EXTRACCIÓN DE DATOS</br>
Lectura de archivos

In [1]:
import pandas as pd
import os
import glob
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
path = os.getcwd() + "\\Datasets\\"
files = glob.glob(path + "/*precios*")

In [3]:
path

'c:\\Users\\eliza\\Desktop\\PI01_Data04\\app\\Datasets\\'

In [4]:
def read_file(path): 
    """Funcion para leer cada df q vienen en distintos formatos, leer varias hojas de un xlsx """
    for i in path:
        ext = os.path.splitext(i)[-1].lower() #deja solo las extensiones
        if ext=='.xlsx':
            xls = pd.ExcelFile(i)
            sheets=xls.sheet_names
            if len(sheets)==1:
                df=pd.read_excel(i)
                return df
            elif len(sheets)==2:
                df1=pd.read_excel(i,sheet_name=0).assign(semana=sheets[0])
                df2=pd.read_excel(i,sheet_name=1).assign(semana=sheets[1])
                return df1, df2
            else:
                print('Archivo Excel con mas de dos sheets')
        elif ext=='.csv':
            df=pd.read_csv(i,sep=',',encoding='utf-16-le').assign(semana=os.path.basename(i).split('.')[0])
            return df

        elif ext=='.txt':
            df=pd.read_csv(i, sep='|').assign(semana=os.path.basename(i).split('.')[0])
            return df

        elif ext=='.json':
            df=pd.read_json(i).assign(semana=os.path.basename(i).split('.')[0])
            return df

        elif ext=='.parquet':
            df=pd.read_parquet(i)
            return df
        else:
            print("I just can't")

In [5]:
#Lectura de archivo
producto = glob.glob(path + "/producto.parquet")
producto = read_file(producto) #con funcion read_file definida arriba

sucursal = glob.glob(path + "/*sucursal*")
sucursal = pd.read_csv(sucursal[0])

precio13 = glob.glob(path + "/*precios_semana_20200413*")
precio13 = read_file(precio13)

precio03 = glob.glob(path + "/*precios_semana_20200503*")
precio03 = read_file(precio03)

precio18 = glob.glob(path + "/*precios_semana_20200518*")
precio18 = read_file(precio18)

precios19 = pd.read_excel(files[0],sheet_name=1).assign(semana="precios_semanas_20200419")
precios26 = pd.read_excel(files[0], sheet_name=0).assign(semana="precios_semanas_20200426")

Transformación y limpieza de df: producto

In [6]:
#Veamos los nulos
producto.isna().sum()*100/72038

id               0.000000
marca            0.002776
nombre           0.002776
presentacion     0.002776
categoria1      99.994447
categoria2      99.994447
categoria3      99.994447
dtype: float64

In [7]:
#Se eliminaran las columnas categoria ya que no aportan informacion relevante
#inplace=True se usa dependiendo de si queremos hacer cambios al df original o no.
producto.drop(columns=['categoria1','categoria2','categoria3'], inplace=True)
producto.head(1)

,id,marca,nombre,presentacion
0,0000000001663,LA ANÓNIMA,Radicheta Atada La Anonima 1 Un,1.0 un


In [8]:
#normalizando id por producto_id
producto["producto_id"] = producto["id"]
producto.drop(columns=["id"], inplace=True)
producto.columns

Index(['marca', 'nombre', 'presentacion', 'producto_id'], dtype='object')

In [9]:
def nasDups(df):
    """Cosas básicas: remover duplicados y nulos"""#servira para load demas dfs
    df = df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    return df
producto = nasDups(producto)

In [10]:
def splitProd(df): #elimina el  - para luego darle tipo de dato a float
    """Algunos productos tenian hardcodeada la sucursal. Removemos esa porcion del string y nos quedamos 
       unicamente con el codigo de producto (12-13 char)"""

    df.producto_id = df.producto_id.str.split('-').str[-1]
    return df
producto = splitProd(producto)
producto.drop_duplicates(subset=['producto_id'], inplace=True) #eliminando duplicados en la columna producto.id ya que seran PK

In [11]:
precio13.dtypes

precio         float64
producto_id     object
sucursal_id     object
semana          object
dtype: object

In [12]:
producto.marca=producto.marca.astype("string")
producto.nombre=producto.nombre.astype("string")
producto.presentacion=producto.presentacion.astype("string")
producto.producto_id=producto.producto_id.astype("int64")

In [13]:
producto.dtypes

marca           string[python]
nombre          string[python]
presentacion    string[python]
producto_id              int64
dtype: object

Transformacion y limpieza de datos de df: de sucursal

In [14]:
#normalizando id por sucursal_id
sucursal["sucursal_id"] = sucursal["id"]
sucursal.drop(columns=["id"], inplace=True)
sucursal.columns

Index(['comercioId', 'banderaId', 'banderaDescripcion', 'comercioRazonSocial',
       'provincia', 'localidad', 'direccion', 'lat', 'lng', 'sucursalNombre',
       'sucursalTipo', 'sucursal_id'],
      dtype='object')

In [15]:
sucursal = nasDups(sucursal)

In [16]:
def unDate(df):
    
    def unDateSeries(x):
        """read_excel me trae algunas sucursal_id como si fueran datetime. Esta funcion toma elementos de una serie, 
           chequea si son datetime y en caso de serlo, los convierte a str con el formato acorde al orden de identificadores de sucursal"""
       
        if isinstance(x,datetime):
            x=x.strftime("%#d-%#m-%Y")
        return x

    df.sucursal_id = df.sucursal_id.apply(unDateSeries)
    return df 

In [17]:
#transformando: eliminando los 00:00:00 de sucursal_id
def delete00(df):
    if df.sucursal_id.dtype != "str":
            df.sucursal_id = df.sucursal_id.astype('string')
    df['sucursal_id'] = df['sucursal_id'].apply(lambda x: x.split(' ')[0])
    return df

In [18]:
sucursal = delete00(sucursal)

In [19]:
def type_sucu(sucursal):
    sucursal.sucursal_id=sucursal.sucursal_id.astype('string')    
    sucursal.comercioId = sucursal.comercioId.astype(int)
    sucursal.banderaId = sucursal.banderaId.astype(int)
    sucursal.banderaDescripcion = sucursal.banderaDescripcion.astype('string')
    sucursal.comercioRazonSocial = sucursal.comercioRazonSocial.astype('string')
    sucursal.provincia = sucursal.provincia.astype('string')
    sucursal.localidad = sucursal.localidad.astype('string')
    sucursal.direccion = sucursal.direccion.astype('string')
    sucursal.sucursalNombre = sucursal.sucursalNombre.astype('string')
    sucursal.sucursalTipo = sucursal.sucursalTipo.astype('string')
    sucursal.columns = sucursal.columns.str.lower()
    return sucursal
sucursal = type_sucu(sucursal)

In [20]:
sucursal.dtypes

comercioid                      int32
banderaid                       int32
banderadescripcion     string[python]
comerciorazonsocial    string[python]
provincia              string[python]
localidad              string[python]
direccion              string[python]
lat                           float64
lng                           float64
sucursalnombre         string[python]
sucursaltipo           string[python]
sucursal_id            string[python]
dtype: object

Tranformación, limpieza, normalizacion dataframe: precios

In [21]:
#Aplico la funcion para eliminar nulos y duplicados
precio13 = nasDups(precio13)
precio03 = nasDups(precio03)
precio18 = nasDups(precio18)
precios19 = nasDups(precios19)
precios26 = nasDups(precios26)

In [22]:
#aplicar la funcion para quitar el caracter especial - y tomar solo numeros
precio13 = splitProd(precio13)
precio03 = splitProd(precio03)
precio18 = splitProd(precio18)
precios19 = splitProd(precios19)

In [23]:
#funcion para quitar 00:00:00 y tomar solo los demas numeros correctos
precio13 = delete00(precio13)
precio03 = delete00(precio03)
precio18 = delete00(precio18)
precios19 = delete00(precios19)
precios26 = delete00(precios26)

In [24]:
def caracter(df):
    if df.precio.dtype != "float":
        """Algunos precios llegan con un string vacio en vez de nulo. Los convertimos a None para removerlos despues. 
           A los que tienen valores, los convertimos a float"""
        try:
            df.precio = df.precio.apply(lambda x: None if x == '' else x)
            df.precio = df.precio.astype(float)
        except:
            print("Error type precios")
    return df.dropna()

In [25]:
#Asignación de los tipos de datos de los valores de los dfs: precios
#algunos ya estan solo los q faltan
def type_precios(df):
    df.semana = df.semana.astype("string")
    df.sucursal_id = df.sucursal_id.astype("string")
    df.producto_id = df.producto_id.astype("int64")
    return df

In [26]:
#aplico la funcion anterior
precio13 = caracter(precio13)
precio03 = caracter(precio03)
precio18 = caracter(precio18)
precios19 = caracter(precios19)
precios26 = caracter(precios26)

In [27]:
#asigno el tipo de dato para df: precio
precio13 = type_precios(precio13)
precio03 = type_precios(precio03)
precio18 = type_precios(precio18)
precios19 = type_precios(precios19)
precios26 = type_precios(precios26)

In [28]:
precios26.dtypes

precio                float64
producto_id             int64
sucursal_id    string[python]
semana         string[python]
dtype: object

In [29]:
#juntar los dfs de los todos los precios con concat
precios = pd.concat([precio13, precio03, precio18, precios19, precios26]) 
#aplico los tipos de datos
precios = type_precios(precios)
#dropeo algunos duplicados
precios = precios.drop_duplicates()

In [30]:
precios.dtypes

precio                float64
producto_id             int64
sucursal_id    string[python]
semana         string[python]
dtype: object

In [31]:
precios.duplicated().sum()

0

voy a unir en un solo df *1ero* en *Python* para luego levantar una API en docker, luego cargar los datos a *MySql* con foreign keys 

In [32]:
# Realizar el inner join
producto_precios = pd.merge(producto, precios, on='producto_id', how='inner')
df = pd.merge(producto_precios, sucursal, on='sucursal_id', how='inner')
#exportamos para asi poder leer desde fastapi

In [33]:
#redusco ala 4 ta parte de os registors ya q pesa demasiado para suboir gratuitamente a github
df = df.iloc[::4]
df.shape
df.to_csv(r'Datasets/df.csv', index=False)

consultas

In [34]:
#query 1: Promedio de precio para sucursal_id '9-1-688': 202.56114869626498
def promedio_precio(sucursal_id:str):
    precio_promedio1 = df[df['sucursal_id'] == sucursal_id]['precio'].mean()
    return {'promedio de precio': precio_promedio1}

promedio_precio('2-1-014')

{'promedio de precio': 172.97052380952383}

In [35]:
#query 2: calcular el precio promedio de la marca DOVE
def precio_promedio_marca(marca:str):
    precio_promedio_marca = df[df['marca'] == marca]['precio'].mean()
    return {"precio promedio de la marca DOVE:": precio_promedio_marca}
precio_promedio_marca('DOVE')

{'precio promedio de la marca DOVE:': 1934.7378714249558}

In [36]:
#query3: cantidad de veces que se vendio el producto_id mas vendido de la sucursaltipo 'Supermercado'
def producto_mas_vendido(Supermercado:str):
    supermercado_df = df[(df['sucursaltipo']==Supermercado)]
    producto_mas_vendido = supermercado_df['producto_id'].value_counts().idxmax()
    cantidad_mas_vendida = supermercado_df['producto_id'].value_counts().max()
    
    return {'producto más vendido': producto_mas_vendido,
            'Cantidad de veces que se vendió':cantidad_mas_vendida}

producto_mas_vendido('Supermercado')

{'producto más vendido': 7791290010482, 'Cantidad de veces que se vendió': 51}